# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [2]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [3]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [4]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

### 10 countries with the lowest infant mortality rates:

In [6]:
import pandas as pd
document = ET.parse( './data/mondial_database.xml' )

root=document.getroot()

In [4]:
inm=[]
for child in root:
    if  child.find('infant_mortality')!=None:
         inm.append([child.find('name').text, child.find('infant_mortality').text])
inmdf=pd.DataFrame(inm)
inmdf.columns=['Country', 'Infant_mortality']
inmdf['Infant_mortality']=inmdf.Infant_mortality.astype(float)

In [5]:
inmdf.sort_values(by='Infant_mortality').head(10)

,Country,Infant_mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


### 10 cities with the biggest population:

In [17]:
popc=[]
popc2=[]

# I had loop through the element in country twice since some cities are directly under country,
# other cities are under country->province->city.  Check in communitiy if there's a simplier way to do it. 
for element in document.iterfind('country'):
        for subelement in element.findall('city'):
            if subelement.find('population')!=None:
                popc.append([subelement.find('name').text, subelement.findall('population')[-1].text])
                
        for subelement in element.findall('./province/city'):
            if subelement.find('population')!=None:
                popc2.append([subelement.find('name').text, subelement.findall('population')[-1].text])        

popc.extend(popc2)

popcdf=pd.DataFrame(popc)
popcdf.columns=['city', 'population']
popcdf['population']=popcdf.population.astype(float)

In [18]:
popcdf.sort_values(by='population', ascending=False).head(10)


,city,population
1587,Shanghai,22315474.0
1068,Istanbul,13710512.0
1748,Mumbai,12442373.0
816,Moskva,11979529.0
1586,Beijing,11716620.0
2698,São Paulo,11152344.0
1588,Tianjin,11090314.0
1310,Guangzhou,11071424.0
1794,Delhi,11034555.0
1313,Shenzhen,10358381.0


### 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [21]:
pop=[]
for element in document.iterfind('country'):
    if element.find('ethnicgroup')!= None:
        b=[]
        c=[]
        a=element.findall('population')[-1].text
        for subele in element.findall('ethnicgroup'):
            b.append(subele.text)
            c.append(subele.attrib['percentage'])
            d=dict(zip(b,c))
        pop.append([a,d])

#Create the list with all the ethnic groups
ls=[]        
for item in pop:
    for k, v in item[1].items():
        item[1][k]=int(float(v)*int(item[0])/100)
    ls.append(item[1])


In [22]:
    
from collections import defaultdict

#Create a dictionary with all the ethnic groups and sum the values
output=defaultdict(int)
for d in ls:
        for k,v in d.items():
           output[k]+=v
    
#Convert the dicontary to dataframe and calculate the top 10

df=pd.DataFrame(output.items(), columns=['ethnic', 'population'])

df.sort_values(by='population', ascending=False).head(10)  

,ethnic,population
93,Han Chinese,1245058800
103,Indo-Aryan,871815583
16,European,494872201
124,African,318325104
178,Dravidian,302713744
136,Mestizo,157734349
183,Bengali,146776916
179,Russian,131856989
269,Japanese,126534212
172,Malay,121993548


### name and country of a) longest river

In [203]:
river=[]
for element in document.iterfind('river'):
    if element.find('length')!=None:
        river.append([element.find('name').text , element.find('length').text, element.attrib['country']])

riverdf=pd.DataFrame(river)
riverdf.columns=['river', 'length', 'country']

riverdf['length']=riverdf['length'].astype(float)

riverdf.sort_values(by='length', ascending=False).head(1)
    

    

,river,length,country
174,Amazonas,6448.0,CO BR PE


### name and country of largest lake 

In [202]:
lake=[]
for element in document.iterfind('lake'):
    if element.find('area')!=None:
        lake.append([element.find('name').text , element.find('area').text, element.attrib['country']])

lakedf=pd.DataFrame(lake)
lakedf.columns=['lake', 'area', 'country']

lakedf['area']=lakedf['area'].astype(float)

lakedf.sort_values(by='area', ascending=False).head(1)

,lake,area,country
54,Caspian Sea,386400.0,R AZ KAZ IR TM


### name and country of airport at highest elevation

In [201]:
airport=[]
for element in document.iterfind('airport'):
    if element.find('elevation')!=None:
        airport.append([element.find('name').text , element.find('elevation').text, element.attrib['country']])

apdf=pd.DataFrame(airport)
apdf.columns=['airport', 'elevation', 'country']

apdf['elevation']=apdf['elevation'].astype(float)

apdf.sort_values(by='elevation', ascending=False).head(1)

,airport,elevation,country
80,El Alto Intl,4063.0,BOL


In [10]:
pwd

u'/Users/coolfish/Documents/Springboard/data_wrangling_xml'